<a href="https://colab.research.google.com/github/difinative/k8-performance-check/blob/main/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless
!pip install pillow


In [ ]:
import cv2
from PIL import Image
from IPython.display import display

image_path = "/content/drive/MyDrive/OCR/IMG-20230902-WA0001.jpg"
img = cv2.imread(image_path)
display(Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))


In [ ]:
mask = np.zeros(img.shape[:2], np.uint8)
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

rect = (10, 10, img.shape[1]-10, img.shape[0]-10)  # Define a rectangle around the object
cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
img_no_bg = img * mask2[:, :, np.newaxis]


In [ ]:
kernel = np.array([[-1,-1,-1],
                   [-1, 9,-1],
                   [-1,-1,-1]])
sharpened = cv2.filter2D(img_no_bg, -1, kernel)


In [ ]:
display(Image.fromarray(cv2.cvtColor(sharpened, cv2.COLOR_BGR2RGB)))


New clustering

In [2]:
!pip install easyocr
!pip install opencv-python
!pip install numpy
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.2 MB/s eta 0:00:00


In [3]:
!pip install torch torchvision


In [13]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models


In [23]:
import cv2
import numpy as np


def preprocess_image(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian or bilateral filtering to reduce noise
    image = cv2.GaussianBlur(image, (5, 5), 0)
    # Alternatively, you can use bilateral filtering:
    # image = cv2.bilateralFilter(image, 9, 75, 75)

    # Binarization to convert the text to black and white for better contrast
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)

    # Remove small artifacts and noise
    kernel = np.ones((3, 3), np.uint8)
    cleaned_image = cv2.erode(binary_image, kernel, iterations=1)

    return cleaned_image

def extract_features(image_path):
    # Preprocess the image
    cleaned_image = preprocess_image(image_path)

    # Load a pre-trained ResNet model
    resnet_model = models.resnet50(pretrained=True)
    # Remove the classification layer (the last layer)
    resnet_model = torch.nn.Sequential(*list(resnet_model.children())[:-1])
    resnet_model.eval()

    # Convert the cleaned image to a 3-channel image
    cleaned_image = cv2.cvtColor(cleaned_image, cv2.COLOR_GRAY2RGB)

    # Resize and normalize the image
    image = transforms.ToPILImage()(cleaned_image)
    image = transforms.Resize((224, 224))(image)
    image = transforms.ToTensor()(image)
    image = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(image)
    image = image.unsqueeze(0)  # Add batch dimension

    # Extract features using the ResNet model
    with torch.no_grad():
        features = resnet_model(image)

    # Convert the feature vector to a list and ensure it's serializable
    feature_vector = features.squeeze().numpy().tolist()

    # Convert any int64 elements to standard Python integers
    feature_vector = [int(x) if isinstance(x, np.int64) else x for x in feature_vector]

    return feature_vector





In [24]:
for image_filename in os.listdir(input_directory):
    image_path = os.path.join(input_directory, image_filename)

    # Extract text using EasyOCR
    annotations = []
    text_results = extract_text(image_path)

    for idx, (text, _, bbox) in enumerate(text_results):
        # Calculate feature vector using ResNet
        feature_vector = extract_features(image_path)
        cluster = "cluster_" + str(idx)  # Assign a temporary cluster label

        # Create annotation dictionary
        annotation = {
            "text": text,
            "cluster": cluster,
            "bbox": bbox.tolist(),
            "Feature Vector": feature_vector

        }
        annotations.append(annotation)

    # Save annotations as JSON
    save_annotations(image_path, annotations)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TypeError: ignored

In [21]:
import os
import json
import numpy as np
from sklearn.cluster import KMeans
import torch
import torchvision.transforms as transforms
import torchvision.models as models

# Custom JSON encoder to handle int64 serialization
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.int64):
            return int(obj)
        return super(NumpyEncoder, self).default(obj)

def load_feature_vectors_from_json(directory):
    feature_vectors = []
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            json_file = os.path.join(directory, filename)
            with open(json_file, 'r') as f:
                data = json.load(f)
                for annotation_list in data.values():
                    for annotation in annotation_list:
                        feature_vector = annotation["Feature Vector"]
                        feature_vectors.append(feature_vector)
    return np.array(feature_vectors)

def cluster_and_update_json_files(input_directory, output_directory, num_clusters):
    feature_vectors = load_feature_vectors_from_json(input_directory)

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(feature_vectors)
    cluster_labels = kmeans.labels_

    # Update the "cluster" attribute in the JSON files
    for filename in os.listdir(input_directory):
        if filename.endswith(".json"):
            json_file = os.path.join(input_directory, filename)
            with open(json_file, 'r') as f:
                data = json.load(f)
                for image_filename, annotation_list in data.items():
                    for annotation in annotation_list:
                        cluster_idx = cluster_labels[0]
                        annotation["cluster"] = f"cluster_{cluster_idx}"

            # Save the updated JSON data to the output directory
            output_file = os.path.join(output_directory, filename)
            with open(output_file, 'w') as f:
                json.dump(data, f, indent=2)

if __name__ == "__main__":
    input_directory = "/content/drive/MyDrive/OCR/output_images"  # Replace with your input directory
    output_directory = "/content/drive/MyDrive/OCR/output_images_data"  # Replace with your output directory
    num_clusters = 500  # Number of clusters

    cluster_and_update_json_files(input_directory, output_directory, num_clusters)


ValueError: ignored

In [22]:
import os
import easyocr
import cv2
import numpy as np
from sklearn.cluster import KMeans

def extract_text(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    return result

def save_annotations(image_path, annotations):
    json_file = os.path.splitext(os.path.basename(image_path))[0] + '.json'
    with open(json_file, 'w') as f:
        json.dump(annotations, f, indent=2)

def main(input_directory, output_directory):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Process each image in the input directory
    for image_filename in os.listdir(input_directory):
        image_path = os.path.join(input_directory, image_filename)

        # Extract text using EasyOCR
        annotations = []
        text_results = extract_text(image_path)

        for idx, (text, _, bbox) in enumerate(text_results):
            # Calculate feature vector (you can replace this with ResNet feature extraction)
            feature_vector = np.random.rand(100)  # Placeholder for feature vector
            cluster = "cluster_" + str(idx)  # Assign a temporary cluster label

            # Create annotation dictionary
            annotation = {
                "text": text,
                "cluster": cluster,
                "bbox": bbox.tolist(),
                "Feature Vector": feature_vector.tolist()
            }
            annotations.append(annotation)

        # Save annotations as JSON
        save_annotations(image_path, annotations)

if __name__ == "__main__":
    input_directory = "/content/drive/MyDrive/OCR/output_images"  # Replace with your input directory
    output_directory = "/content/drive/MyDrive/OCR/output_images_data"  # Replace with your output directory
    main(input_directory, output_directory)


TypeError: ignored

In [7]:
import os
import json
import numpy as np
from sklearn.cluster import KMeans

def load_feature_vectors_from_json(directory):
    feature_vectors = []
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            json_file = os.path.join(directory, filename)
            with open(json_file, 'r') as f:
                data = json.load(f)
                for annotation in data.values():
                    feature_vector = np.array(annotation["Feature Vector"])
                    feature_vectors.append(feature_vector)
    return np.array(feature_vectors)

def cluster_and_update_json_files(input_directory, output_directory, num_clusters):
    feature_vectors = load_feature_vectors_from_json(input_directory)

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(feature_vectors)
    cluster_labels = kmeans.labels_

    # Update the "cluster" attribute in the JSON files
    for filename in os.listdir(input_directory):
        if filename.endswith(".json"):
            json_file = os.path.join(input_directory, filename)
            with open(json_file, 'r') as f:
                data = json.load(f)
                for image_filename, annotation_list in data.items():
                    for annotation in annotation_list:
                        cluster_idx = cluster_labels[0]
                        annotation["cluster"] = f"cluster_{cluster_idx}"

            # Save the updated JSON data to the output directory
            output_file = os.path.join(output_directory, filename)
            with open(output_file, 'w') as f:
                json.dump(data, f, indent=2)

if __name__ == "__main__":
    input_directory = "/content/drive/MyDrive/OCR/output_images"  # Replace with your input directory
    output_directory = "/content/drive/MyDrive/OCR/output_images_data"  # Replace with your output directory
    num_clusters = 500  # Number of clusters

    cluster_and_update_json_files(input_directory, output_directory, num_clusters)


ValueError: ignored

In [5]:
!rm /content/drive/MyDrive/OCR/output_images/*

In [32]:
import cv2
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
import json

# Define a list of allowed image extensions
allowed_image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

def preprocess_image(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian or bilateral filtering to reduce noise
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # Binarization to convert the text to black and white for better contrast
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)

    # Remove small artifacts and noise
    kernel = np.ones((3, 3), np.uint8)
    cleaned_image = cv2.erode(binary_image, kernel, iterations=1)

    return cleaned_image

def extract_features(image_path):
    # Preprocess the image
    cleaned_image = preprocess_image(image_path)

    # Load a pre-trained ResNet model
    resnet_model = models.resnet50(pretrained=True)
    # Remove the classification layer (the last layer)
    resnet_model = torch.nn.Sequential(*list(resnet_model.children())[:-1])
    resnet_model.eval()

    # Convert the cleaned image to a 3-channel image
    cleaned_image = cv2.cvtColor(cleaned_image, cv2.COLOR_GRAY2RGB)

    # Resize and normalize the image
    image = transforms.ToPILImage()(cleaned_image)
    image = transforms.Resize((224, 224))(image)
    image = transforms.ToTensor()(image)
    image = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(image)
    image = image.unsqueeze(0)  # Add batch dimension

    # Extract features using the ResNet model
    with torch.no_grad():
        features = resnet_model(image)

    # Convert the feature vector to a list
    feature_vector = features.squeeze().numpy()

    return feature_vector

def save_annotations(image_path, annotations):
    # Construct the output JSON file path based on the input image filename
    image_filename = os.path.basename(image_path)
    output_json_filename = os.path.splitext(image_filename)[0] + '.json'
    output_json_path = os.path.join(output_directory, output_json_filename)

    # Serialize the annotations to JSON using the custom encoder
    with open(output_json_path, 'w') as json_file:
        json.dump(annotations, json_file, cls=NumpyEncoder)

input_directory = '/content/drive/MyDrive/OCR/output_images'
output_directory = '/content/drive/MyDrive/OCR/output_images_data'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Process each image in the input directory
for image_filename in os.listdir(input_directory):
    # Check if the file has an allowed image extension
    _, file_extension = os.path.splitext(image_filename)
    if file_extension.lower() not in allowed_image_extensions:
        print(f"Skipping file '{image_filename}' as it's not an allowed image.")
        continue

    image_path = os.path.join(input_directory, image_filename)

    # Extract text using EasyOCR
    annotations = []  # Store annotations for this image
    text_results = extract_text(image_path)

    for idx, (text, _, bbox) in enumerate(text_results):
        # Calculate feature vector using ResNet
        feature_vector = extract_features(image_path)

        cluster = "cluster_" + str(idx)  # Assign a temporary cluster label

        # Create annotation dictionary
        annotation = {
            "text": text,
            "cluster": cluster,
            "bbox": bbox.tolist(),
            "Feature Vector": feature_vector
        }
        annotations.append(annotation)

    # Save annotations as JSON
    save_annotations(image_path, annotations)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Skipping file '.ipynb_checkpoints' as it's not an allowed image.


In [31]:
!rm /content/drive/MyDrive/OCR/output_images_data/*